In [1]:
# To run locally, uncomment the below
# ! pip install -e ../

# To run in Onyx, uncomment the below
# import sys
# sys.path.append("..")

Obtaining file:///home/jdownes/onyx-genai-devkit
  Attempting uninstall: onyxgenai
    Found existing installation: onyxgenai 0.1
    Uninstalling onyxgenai-0.1:
      Successfully uninstalled onyxgenai-0.1
  Running setup.py develop for onyxgenai


In [2]:
from onyxgenai.extract import extract_text_from_folder, tokenize_sentences
from onyxgenai.embed import Embedder
from onyxgenai.generate import TextGenerator
from onyxgenai.model import ModelStore

# Extract text from mixed document types

In [3]:
text,files = extract_text_from_folder("../data")
all_sentences = tokenize_sentences(text) 
sentences = [s for ss in all_sentences for s in ss] #flatten

# Embed and persist this data for RAG

In [4]:
embedding_service = Embedder("http://embed.onyx-services/", model="all-MiniLM-L6-v2", model_version=8, num_workers=4, collection_name="Sample_RAG_Demo")
embeddings= embedding_service.embed_text(sentences)


embedding 55 items


# Start RAG/GenAI services
This allows for realtime usage and persistent service

In [5]:
embedding_service_id = embedding_service.start_service( n_workers=1)

generator_service = TextGenerator("http://generate.onyx-services/",model="Mistral-Instruct-7B")
generator_service_id = generator_service.start_service()

# Execute a RAG workflow

In [6]:
prompt = "What ticketing tools has DHS utilized for IT services?"
answer = generator_service.generate(prompt,context=embedding_service.query_text(prompt) )


In [7]:
print(answer)

This is sample response from Mistral-Instruct-7B, with prompt of 'What ticketing tools has DHS utilized for IT services?' with context of ['This is result 0', 'This is result 1', 'This is result 2', 'This is result 3', 'This is result 4']


# Shutdown the services

In [8]:
generator_service.stop_service()
embedding_service.stop_service()


# Get List of Available Models

In [ ]:
model_service = ModelStore("http://store.onyx-services/")
models = model_service.get_models()
print(models)